<a href="https://colab.research.google.com/github/Yundo37/automation-portfolio-Yundo/blob/main/automation_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 환경 세팅 (Colab 전용)
# Colab 초기화: 이전 실행 내용 제거 + 필수 폴더 재생성
!rm -rf pageObjects utilities testData tests reports temp_repo
!mkdir -p pageObjects utilities testData tests reports

# Chrome + 필요한 패키지 설치
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install
!pip install -q selenium pytest pytest-html webdriver-manager

# GitHub에서 모듈 코드만 가져오기 (temp_repo 사용)
!git clone https://github.com/Yundo37/automation-portfolio-Yundo.git temp_repo
!cp -r temp_repo/pageObjects/* pageObjects/
!cp -r temp_repo/utilities/* utilities/
!cp -r temp_repo/testData/* testData/
!cp    temp_repo/tests/conftest.py tests/
!rm -rf temp_repo

# __init__.py 생성으로 패키지 인식
for d in ["pageObjects", "utilities", "testData", "tests"]:
    open(f"{d}/__init__.py", "w").close()

# sys.path에 경로 등록
import sys, os
sys.path += [os.path.abspath(p) for p in [".", "pageObjects", "utilities", "testData", "tests"]]

# 간단한 점검
import importlib.util
print("== 코드 임포트 확인 ==")
print("pageObjects.HomePage:", importlib.util.find_spec("pageObjects.HomePage") is not None)
print("utilities.BaseClass:", importlib.util.find_spec("utilities.BaseClass") is not None)
print("testData.HomePageData:", importlib.util.find_spec("testData.HomePageData") is not None)
print("\n✅ 초기화 완료")

In [ ]:
# 2. tests/test_e2e.py
%%writefile tests/test_e2e.py
import pytest
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pageObjects.HomePage import HomePage

@pytest.mark.usefixtures("setup")
class TestOne:

    def test_e2e(self):
        print("E2E 테스트 시작")
        homePage = HomePage(self.driver)
        checkoutPage = homePage.shopItems()
        print("상품 목록 조회")

        products = checkoutPage.getProductTitles()
        for product in products:
            productName = product.find_element(By.XPATH, "div/h4/a").text
            print("상품명:", productName)
            if productName == "Blackberry":
                checkoutPage.getProductFooter(product).click()
                print("Blackberry 장바구니에 담기 완료")
                break

        # ✅ checkout 버튼 수동 클릭 (JS + scrollIntoView)
        wait = WebDriverWait(self.driver, 10)
        cart_btn = self.driver.find_element(By.CSS_SELECTOR, "a[class*=btn-primary]")
        self.driver.execute_script("arguments[0].scrollIntoView(true);", cart_btn)
        time.sleep(1)
        self.driver.execute_script("arguments[0].click();", cart_btn)
        print("장바구니 페이지로 이동")

        # ✅ checkoutpage.CheckOutItems() → btn-success 클릭
        confirmPage = checkoutPage.CheckOutItems()
        print("결제 페이지로 이동")

        confirmPage.enterCountry("India")
        self.verifyLinkPresence("India")
        confirmPage.selectCountry().click()
        confirmPage.checkTerms().click()
        confirmPage.submitOrder().click()

        successText = confirmPage.getSuccessMessage()
        print("최종 결과:", successText)
        assert "Success" in successText

    def verifyLinkPresence(self, text):
        wait = WebDriverWait(self.driver, 10)
        wait.until(EC.presence_of_element_located((By.LINK_TEXT, text)))

In [ ]:
# 3. tests/test_HomePage.py
%%writefile tests/test_HomePage.py
import pytest

from pageObjects.HomePage import HomePage
from testData.HomePageData import HomePageData
from utilities.BaseClass import BaseClass

class TestHomePage(BaseClass):

    def test_formSubmission(self, getData):
        print("폼 제출 테스트 시작")
        homepage = HomePage(self.driver)
        print("입력 데이터:", getData)

        homepage.getName().send_keys(getData["username"])
        homepage.getEmail().send_keys(getData["email"])
        homepage.getPassword().send_keys("123456")
        homepage.selectCheckbox().click()
        homepage.selectEmploymentStatus().click()

        self.selectOptionByText(homepage.getGenderDropdown(), getData["gender"])
        self.selectOptionByIndex(homepage.getGenderDropdown(), 0)

        homepage.submitForm().click()
        message = homepage.getSuccessMessage()
        print("폼 제출 결과:", message)

        # ✅ 의도적 실패 1건만 발생: 첫 데이터셋(username == "YB")만 실패
        if getData.get("username") == "YB":
            assert "Success123" in message  # 일부러 실패
        else:
            assert "Success" in message     # 정상 검증

        self.driver.refresh()

    @pytest.fixture(params=HomePageData.test_Homepage_data)
    def getData(self, request):
        return request.param

In [ ]:
# 4. 테스트 실행 + HTML 리포트 실시간 출력 보기

import sys
import os
sys.path.append(os.path.abspath("."))  # 현재 Colab 작업 디렉토리 등록

import subprocess
from IPython.display import display, HTML

print("========== [테스트 실행 로그 출력] ==========")

# ✅ 실시간 출력 위해 Popen 사용
with subprocess.Popen(
    ["pytest", "-s", "tests/test_e2e.py", "tests/test_HomePage.py", "--html=reports/report.html", "--self-contained-html"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
) as proc:
    for line in proc.stdout:
        print(line, end="")  # 실시간 출력

# ✅ HTML 리포트 Colab에서 바로 보기
with open("reports/report.html", "r", encoding="utf-8") as f:
    html_content = f.read()
display(HTML(html_content))